In [1]:
cd "C:\Users\drn455\Documents\GitHub\SQuID-Lab-quam"

C:\Users\drn455\Documents\GitHub\SQuID-Lab-quam


In [2]:
# cd C:\Users\jacob\OneDrive\Documents\GitHub\SQuID-Lab-quam

In [3]:
# cd C:\Users\MESO 1\Documents\GitHub\SQuID-Lab-quam

In [4]:
from squid_lab_quam.components.roots import QuAMSCQ1

machine = QuAMSCQ1()

2024-07-03 16:32:57,356 - qm - INFO     - Starting session: bf297f1b-1f80-468b-be7f-6064aad46449


## Add information

In [5]:
from squid_lab_quam.components.information import Information

information = Information(
    user_name="Jacob Hastrup",
    user_ku_tag="drn455",
    device_name="Chalmers/5q4_v0",
    fridge_name="meso",
    data_path=r"\\unicph.domain\groupdir\SCI-NBI-QDev\SQuID Lab Data\Devices\Chalmers 5q4c v1.1\OPX Data",
    # state_path=r"C:\Users\MESO 1\Documents\GitHub\SQuID-Lab-quam\squid_lab_quam\tests",
    state_path=r"C:\Users\jacob\OneDrive\Documents\GitHub\SQuID-Lab-quam\squid_lab_quam\tests\state",
)

machine.information = information

information.print_info()

User name: Jacob Hastrup (drn455)
Device name: Chalmers/5q4_v0
Fridge name: meso


## Add network

In [6]:
from squid_lab_quam.components.network import OPXNetwork, OctaveNetwork

network = OPXNetwork(
    host="10.209.67.168",
    cluster_name="Cluster_1",
    octave_networks={
        "octave1": OctaveNetwork(
            octave_host="10.209.67.174", octave_port=80, controller="con1"
        )
    },
)

machine.network = network

## Add wiring

In [7]:
from squid_lab_quam.components.wiring import (
    OPXWiring,
    OPXFeedLineWiring,
    OPXIQChannelWiring,
)

wiring = OPXWiring(
    drive_lines={
        "q0": OPXIQChannelWiring(
            port_I=("con1", 0),
            port_Q=("con1", 0),
        ),
        "q1": OPXIQChannelWiring(
            port_I=("con1", 0),
            port_Q=("con1", 0),
        ),
        "q2": OPXIQChannelWiring(
            port_I=("con1", 0),
            port_Q=("con1", 0),
        ),
        "q3": OPXIQChannelWiring(
            port_I=("con1", 0),
            port_Q=("con1", 0),
        ),
        "q4": OPXIQChannelWiring(
            port_I=("con1", 0),
            port_Q=("con1", 0),
        ),
    },
    feed_lines={
        "feedline": OPXFeedLineWiring(
            output_I=("con1", 0),
            output_Q=("con1", 0),
            input_I=("con1", 0),
            input_Q=("con1", 0),
        )
    },
)

machine.wiring = wiring

## Add octaves

In [8]:
from squid_lab_quam.components.octave import OctaveSQuID

octaves = {
    "octave1": OctaveSQuID(
        ip=network.octave_networks["octave1"].get_reference("octave_host"),
        port=network.octave_networks["octave1"].get_reference("octave_port"),
    )
}
machine.octaves = octaves

## Add readout frequency converters

In [9]:
from squid_lab_quam.components.octave import (
    OctaveUpConverterSQuID,
    OctaveDownConverterSQuID,
)


readout_up_converter = OctaveUpConverterSQuID()


octaves["octave1"].RF_outputs[1] = readout_up_converter


readout_down_converter = OctaveDownConverterSQuID(
    LO_frequency=readout_up_converter.get_reference("LO_frequency"),
)


octaves["octave1"].RF_inputs[1] = readout_down_converter

## Add qubits

In [10]:
from squid_lab_quam.components.qubits import Transmon
from squid_lab_quam.components.resonators import ReadoutResonator
from quam.components.channels import IQChannel, InOutIQChannel
from squid_lab_quam.components.pulse_sets import PulseSetDragGaussian, PulseSetFlattopCosine
from quam.components.pulses import SquareReadoutPulse

qubit_names = ["q0", "q1", "q2", "q3", "q4"]

machine.shared_qubit_parameters = {
    "drag_gaussian_pulse_parameters": {"length": 40, "sigma": 7, "subtracted": True},
}

for idx, qubit_name in enumerate(qubit_names):
    # Add driving frequency converter
    drive_up_converter = OctaveUpConverterSQuID()
    octaves["octave1"].RF_outputs[idx + 2] = drive_up_converter

    # Add qubit
    qubit = Transmon(
        xy=IQChannel(
            opx_output_I=f"#/wiring/drive_lines/{qubit_name}/port_I",
            opx_output_Q=f"#/wiring/drive_lines/{qubit_name}/port_Q",
            frequency_converter_up=drive_up_converter.get_reference(),
        ),
        resonator=ReadoutResonator(
            channel=InOutIQChannel(
                opx_input_I=f"#/wiring/feed_lines/feedline/input_I",
                opx_input_Q=f"#/wiring/feed_lines/feedline/input_Q",
                opx_output_I=f"#/wiring/feed_lines/feedline/output_I",
                opx_output_Q=f"#/wiring/feed_lines/feedline/output_Q",
                frequency_converter_up=readout_up_converter.get_reference(),
                frequency_converter_down=readout_down_converter.get_reference(),
            )
        ),
        transition_frequencies=[6, 5],
    )
    qubit.pulse_sets = {
        "drag_gaussian": PulseSetDragGaussian(
            amplitude_180=0.5,
            amplitude_90=0.25,
            sigma="#/shared_qubit_parameters/drag_gaussian_pulse_parameters/sigma",
            length="#/shared_qubit_parameters/drag_gaussian_pulse_parameters/length",
            subtracted="#/shared_qubit_parameters/drag_gaussian_pulse_parameters/subtracted",
            anharmonicity="#../../anharmonicity",
        ),
        "flattop_cosine": PulseSetFlattopCosine(
            amplitude=0.1,
            rise_fall_time=16,
        )
    }

    machine.qubits[qubit_name] = qubit

    qubit.resonator.channel.operations["readout"] = SquareReadoutPulse(
        length=1000, amplitude=0.1, threshold=0.0, digital_marker="ON"
    )

In [11]:
machine.qubits["q0"].pulse_sets["drag_gaussian"]._add_drive_pulses()

In [12]:
machine.set_default_gate_shape("drag_gaussian")

In [13]:
machine.save()

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\jacob\\OneDrive\\Documents\\GitHub\\SQuID-Lab-quam\\squid_lab_quam\\tests\\state'